In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.compose import ColumnTransformer
import pickle

In [287]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [288]:
col_with_missing = [col for col in df.columns if df[col].isnull().sum()>0]
print(col_with_missing)

[]


In [289]:
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [290]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [291]:
df["Geography"].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [292]:
#divide into dependent and independent features
X = df.drop('Exited',axis=1)
y = df['Exited']

In [293]:
oneHot_features = ['Geography']
LabelEncoder_features = ['Gender']
numeric_features = [col for col in X.columns if col not in oneHot_features + LabelEncoder_features]

In [294]:
numeric_features

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [295]:
label_encoder_gender = LabelEncoder()
 

In [296]:
preprocessor = ColumnTransformer([
        ('OneHotEncoder', OneHotEncoder(sparse_output=False), oneHot_features),
        ('StandardScaler', StandardScaler(), numeric_features)
    ], remainder='passthrough',
    verbose_feature_names_out=False# to avoid prefixing feature names like remainer__Exited
)

In [297]:
#trian test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [298]:
X_train['Gender'] = label_encoder_gender.fit_transform(X_train['Gender']) 
X_test['Gender'] = label_encoder_gender.transform(X_test['Gender'])

In [299]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)
column_names = preprocessor.get_feature_names_out()
column_names

array(['Geography_France', 'Geography_Germany', 'Geography_Spain',
       'CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Gender'],
      dtype=object)

In [302]:
X_train=pd.DataFrame(X_train,columns=column_names)
X_test=pd.DataFrame(X_test,columns=column_names)

In [303]:
X_train

,Geography_France,Geography_Germany,Geography_Spain,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender
0,1.0,0.0,0.0,0.356500,-0.655786,0.345680,-1.218471,0.808436,0.649203,0.974817,1.367670,1.0
1,0.0,1.0,0.0,-0.203898,0.294938,-0.348369,0.696838,0.808436,0.649203,0.974817,1.661254,1.0
2,0.0,0.0,1.0,-0.961472,-1.416365,-0.695393,0.618629,-0.916688,0.649203,-1.025834,-0.252807,1.0
3,1.0,0.0,0.0,-0.940717,-1.131148,1.386753,0.953212,-0.916688,0.649203,-1.025834,0.915393,0.0
4,1.0,0.0,0.0,-1.397337,1.625953,1.386753,1.057449,-0.916688,-1.540351,-1.025834,-1.059600,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,1.0,0.0,0.0,1.207474,1.435808,1.039728,-0.102301,-0.916688,0.649203,0.974817,-0.539860,1.0
7996,1.0,0.0,0.0,0.314989,1.816097,-1.389442,-1.218471,-0.916688,0.649203,0.974817,-1.733882,0.0
7997,1.0,0.0,0.0,0.865009,-0.085351,-1.389442,-1.218471,2.533560,-1.540351,-1.025834,-0.142765,0.0
7998,1.0,0.0,0.0,0.159323,0.390011,1.039728,1.827259,-0.916688,0.649203,-1.025834,-0.050826,1.0


In [304]:
## save the label encoder
with open('label_encoder_gender.pkl','wb') as f:
    pickle.dump(label_encoder_gender,f)

### save the preprocessor
with open('preprocessor.pkl','wb') as f:
    pickle.dump(preprocessor,f)

In [306]:
X_train.shape, X_test.shape

((8000, 12), (2000, 12))

ANN Implimentation 

In [307]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [308]:
(X_train.shape[1],)

(12,)

In [310]:
## Build Our ANN Model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## HL1 Connected wwith input layer
    Dense(32,activation='relu'), ## HL2
    Dense(1,activation='sigmoid')  ## output layer
])

In [311]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [312]:
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [313]:
loss

In [314]:
## compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [315]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [316]:
## Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [317]:
### Train the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 3s 5ms/step - loss: 0.3849 - accuracy: 0.8407 - val_loss: 0.3617 - val_accuracy: 0.8585
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3516 - accuracy: 0.8577 - val_loss: 0.3424 - val_accuracy: 0.8595
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3456 - accuracy: 0.8584 - val_loss: 0.3478 - val_accuracy: 0.8555
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3431 - accuracy: 0.8574 - val_loss: 0.3405 - val_accuracy: 0.8585
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3425 - accuracy: 0.8626 - val_loss: 0.3501 - val_accuracy: 0.8570
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3363 - accuracy: 0.8595 - val_loss: 0.3510 - val_accuracy: 0.8590
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3338 - accuracy: 0.8615 - val_loss: 0.3464 - val_accuracy: 0.85

In [318]:
model.save('model.h5')

c:\Users\sarth\OneDrive\Documents\MachineLearning Projects\Deep Learning Churn Detection\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [319]:
## Load Tensorboard Extension
%load_ext tensorboard

In [321]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 33668), started 0:04:21 ago. (Use '!kill 33668' to kill it.)